# SVD between a Model and Satellite data

This notebook shows how to multiply two matrices and calculate SVD. Each matrix is created out a set of GeoTiffs for a series of years. Both matrices should have the same dimension.

For demonstration we will use from a model (spring-index) and from a satellite (AVHRR).

## Dependencies

In [1]:
import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}

import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.hadoop.io.SequenceFile.Writer
import org.apache.hadoop.io.{SequenceFile, _}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.{BlockMatrix, CoordinateMatrix, MatrixEntry, RowMatrix}
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}
import org.apache.spark.rdd.RDD
import org.apache.spark.storage.StorageLevel
import org.apache.spark.{SparkConf, SparkContext}

import scala.sys.process._

## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store protected_extent and num_cols_rows into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **all** which are a multi-band (**8 bands**) GeoTiffs
* Or choose single band ones:
    0. Onset_Greenness_Increase
    1. Onset_Greenness_Maximum
    2. Onset_Greenness_Decrease
    3. Onset_Greenness_Minimum
    4. NBAR_EVI_Onset_Greenness_Minimum
    5. NBAR_EVI_Onset_Greenness_Maximum
    6. NBAR_EVI_Area
    7. Dynamics_QC

<span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
var model_rdd_offline_mode = true
var model_matrix_offline_mode = true
var satellite_rdd_offline_mode = true
var satellite_matrix_offline_mode = true

//Using spring-index model
var model_path = "hdfs:///user/hadoop/spring-index/"
var model_dir = "BloomFinal"

//Using AVHRR Satellite data
var satellite_path = "hdfs:///user/hadoop/avhrr/"
var satellite_dir = "SOST"

var out_path = "hdfs:///user/pheno/svd/"
var band_num = 3

//Years between (inclusive) 1989 - 2014
var satellite_first_year = 1989
var satellite_last_year = 2014

//Years between (inclusive) 1980 - 2015
var model_first_year = 1989
var model_last_year = 2014

//Mask
val toBeMasked = true
val mask_path = "hdfs:///user/hadoop/usa_mask.tif"

val save_rdds = true
val save_matrix = true

model_rdd_offline_mode = true
model_matrix_offline_mode = true
satellite_rdd_offline_mode = true
satellite_matrix_offline_mode = true
model_path = hdfs:///user/hadoop/spring-index/
model_dir = BloomFinal
satellite_path = hdfs:///user/hadoop/avhrr/
satellite_dir = SOST
out_path = hdfs:///user/pheno/svd/
band_num = 3
satellite_first_year = 1989
satellite_last_year = 2014
model_first_year = 1989
model_last_year = 2014
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask.tif
save_rdds = true
save_matrix = true


true


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
  mask_str = "_mask"
var model_grid0_path = out_path + model_dir + "_grid0"
var model_grid0_index_path = out_path + model_dir + "_grid0_index"

var model_grid_path = out_path + model_dir + "_grid"
var satellite_grid_path = out_path + satellite_dir + "_grid"
var model_matrix_path = out_path + model_dir + "_matrix"
var satellite_matrix_path = out_path + satellite_dir + "_matrix"
var metadata_path = out_path + model_dir + "_metadata"

val model_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_grid_path))
val model_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(model_matrix_path))
val satellite_rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_grid_path))
val satellite_matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(satellite_matrix_path))

if (model_rdd_offline_mode != model_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_rdd_offline_exists.toString())
  model_rdd_offline_mode = model_rdd_offline_exists
}

if (model_matrix_offline_mode != model_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + model_matrix_offline_exists.toString())
  model_matrix_offline_mode = model_matrix_offline_exists
}

var model_skip_rdd = false
if (model_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    model_skip_rdd = true
}

if (satellite_rdd_offline_mode != satellite_rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_rdd_offline_exists.toString())
  satellite_rdd_offline_mode = satellite_rdd_offline_exists
}

if (satellite_matrix_offline_mode != satellite_matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + satellite_matrix_offline_exists.toString())
  satellite_matrix_offline_mode = satellite_matrix_offline_exists
}

var satellite_skip_rdd = false
if (satellite_matrix_offline_exists) {
    println("Since we have a matrix, the load of the grids RDD will be skipped!!!")
    satellite_skip_rdd = true
}

var corr_tif = out_path + "_" + satellite_dir + "_" + model_dir + ".tif"
var corr_tif_tmp = "/tmp/svd_" + satellite_dir + "_" + model_dir + ".tif"

//Years
val model_years = 1980 to 2015
val satellite_years = 1989 to 2014

if (!satellite_years.contains(satellite_first_year) || !(satellite_years.contains(satellite_last_year))) {
  println("Invalid range of years for " + satellite_dir + ". I should be between " + satellite_first_year + " and " + satellite_last_year)
  System.exit(0)
}

if (!model_years.contains(model_first_year) || !(model_years.contains(model_last_year))) {
  println("Invalid range of years for " + model_dir + ". I should be between " + model_first_year + " and " + model_last_year)
  System.exit(0)
}

if ( ((satellite_last_year - model_first_year) > (model_last_year - model_first_year)) || ((satellite_last_year - model_first_year) > (model_last_year - model_first_year))) {
  println("The range of years for each data set should be of the same length.");
  System.exit(0)
}

var model_years_range = (model_years.indexOf(model_first_year), model_years.indexOf(model_last_year))
var satellite_years_range = (satellite_years.indexOf(satellite_first_year), satellite_years.indexOf(satellite_last_year))

//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var model_grid0: RDD[(Long, Double)] = sc.emptyRDD
var model_grid0_index: RDD[Long] = sc.emptyRDD
var grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var model_grids: RDD[Array[Double]] = sc.emptyRDD
val rows = sc.parallelize(Array[Double]()).map(m => Vectors.dense(m))
var model_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var model_std :Array[Double] = new Array[Double](0)

var satellite_grids_RDD: RDD[Array[Double]] = sc.emptyRDD
var satellite_grids: RDD[Array[Double]] = sc.emptyRDD
var satellite_summary: MultivariateStatisticalSummary = new RowMatrix(rows).computeColumnSummaryStatistics()
var satellite_std :Array[Double] = new Array[Double](0)

var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var satellite_cells_size :Long = 0
var model_cells_size :Long = 0
var t0 : Long = 0
var t1 : Long = 0

Waiting for a Spark session to start...

"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false


conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-504629498_36, ugi=pheno (auth:SIMPLE)]]
mask_str = _mask
model_grid0_path = hdfs:///user/pheno/svd/BloomFinal_grid0
model_grid0_index_path = hdfs:///user/pheno/svd/BloomFinal_grid0_index
model_grid_path = hdfs:///user/pheno/svd/BloomFinal_grid
satellite_grid_path = hdfs:///user/pheno/svd/SOST_grid
model_matrix_path = hdfs:///user/pheno/svd/BloomFinal_matrix
satellite_matrix_path = hdfs:///us...


hdfs:///user/pheno/svd/SOST_matrix

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [5]:
t0 = System.nanoTime()

//Load Mask
if (toBeMasked) {
  val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
  val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
  mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).filter(m => !m._1.isNaN).values.collect())(0)
}

//Local variables
val pattern: String = "tif"
val satellite_filepath: String = satellite_path + satellite_dir
val model_filepath: String = model_path + "/" + model_dir

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 4206005010ns


t0 = 2860612813767
pattern = tif
satellite_filepath = hdfs:///user/hadoop/avhrr/SOST
model_filepath = hdfs:///user/hadoop/spring-index//BloomFinal
t1 = 2864818818777


2864818818777

### Satellite data

In [6]:
t0 = System.nanoTime()

if (satellite_rdd_offline_mode) {
  satellite_grids_RDD = sc.objectFile(satellite_grid_path)
} else {
  //Lets load MODIS Singleband GeoTiffs and return RDD just with the tiles.
  val satellite_geos_RDD = sc.hadoopGeoTiffRDD(satellite_filepath, pattern)
  val satellite_tiles_RDD = satellite_geos_RDD.values

  if (toBeMasked) {
    val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
    satellite_grids_RDD = satellite_tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble().filter(_ != -1000))
  } else {
    satellite_grids_RDD = satellite_tiles_RDD.map(m => m.toArrayDouble())
  }

  //Store in HDFS
  if (save_rdds) {
      satellite_grids_RDD.saveAsObjectFile(satellite_grid_path)
  }
}
val satellite_grids_withIndex = satellite_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
satellite_grids = satellite_grids_withIndex.filterByRange(satellite_years_range._1, satellite_years_range._2).values
satellite_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
satellite_summary = Statistics.colStats(satellite_grids.map(m => Vectors.dense(m)))
satellite_std = satellite_summary.variance.toArray.map(m => scala.math.sqrt(m))

var satellite_grid0_index: RDD[Double] = satellite_grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m)
satellite_cells_size = satellite_grid0_index.count().toInt
println("Number of cells is: " + satellite_cells_size)

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Number of cells is: 10115631                                                    
Elapsed time: 60299547554ns


t0 = 2866874788958
satellite_grids_withIndex = MapPartitionsRDD[28] at map at <console>:103
satellite_grids = MapPartitionsRDD[30] at values at <console>:106
satellite_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@7ba42fb2
satellite_std = [D@77b2ca2e
satellite_grid0_index = MapPartitionsRDD[38] at flatMap at <console>:113
satellite_cells_size = 10115631
t1 = 2927174336512


2927174336512

### Model data

In [7]:
t0 = System.nanoTime()

if (model_rdd_offline_mode) {
  model_grids_RDD = sc.objectFile(model_grid_path)
  model_grid0 = sc.objectFile(model_grid0_path)
  model_grid0_index = sc.objectFile(model_grid0_index_path)
  val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
  projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
  num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
  cellT = deserialize(metadata(3)).asInstanceOf[CellType]
} else {
  val model_geos_RDD = sc.hadoopMultibandGeoTiffRDD(model_filepath, pattern)
  val model_tiles_RDD = model_geos_RDD.values

  //Retrieve the number of cols and rows of the Tile's grid
  val tiles_withIndex = model_tiles_RDD.zipWithIndex().map { case (v, i) => (i, v) }
  val tile0 = (tiles_withIndex.filter(m => m._1 == 0).values.collect()) (0)
  num_cols_rows = (tile0.cols, tile0.rows)
  cellT = tile0.cellType

  //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
  val projected_extents_withIndex = model_geos_RDD.keys.zipWithIndex().map { case (e, v) => (v, e) }
  projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

  val band_numB: Broadcast[Int] = sc.broadcast(band_num)
  if (toBeMasked) {
    val mask_tile_broad: Broadcast[Tile] = sc.broadcast(mask_tile0)
    grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
  } else {
    grids_RDD = model_tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
  }

  //Get Index for each Cell
  val grids_withIndex = grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
  if (toBeMasked) {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
  } else {
    model_grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
  }

  //Get the Tile's grid
  model_grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

  //Lets filter out NaN
  if (toBeMasked) {
    model_grids_RDD = grids_RDD.map(m => m.filter(m => m != -1000.0))
  } else {
    model_grids_RDD = grids_RDD
  }

  //Store data in HDFS
  model_grids_RDD.saveAsObjectFile(model_grid_path)
  model_grid0.saveAsObjectFile(model_grid0_path)
  model_grid0_index.saveAsObjectFile(model_grid0_index_path)

  val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
    Writer.file(metadata_path),
    Writer.keyClass(classOf[IntWritable]),
    Writer.valueClass(classOf[BytesWritable])
  )

  writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
  writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
  writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
  writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
  writer.hflush()
  writer.close()
}

val model_grids_withIndex = model_grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }

//Filter out the range of years:
model_grids = model_grids_withIndex.filterByRange(model_years_range._1, model_years_range._2).values
model_grids.persist(StorageLevel.DISK_ONLY)

//Collect Stats:
model_summary = Statistics.colStats(model_grids.map(m => Vectors.dense(m)))
//model_std = model_summary.variance.toArray.map(m => scala.math.sqrt(m))

var model_tile0_index: RDD[Double] = model_grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m)
model_cells_size = model_tile0_index.count().toInt

t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 115249064459ns                                                    


t0 = 2931208269766
model_grids_withIndex = MapPartitionsRDD[48] at map at <console>:160
model_grids = MapPartitionsRDD[50] at values at <console>:163
model_summary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@2123727a
model_tile0_index = MapPartitionsRDD[58] at flatMap at <console>:170
model_cells_size = 10115631
t1 = 3046457334225


3046457334225

## Matrices

### Satellite

In [16]:
//Satellite
val satellite_cells_sizeB = sc.broadcast(satellite_cells_size)
val satellite_mat: RowMatrix = new RowMatrix(satellite_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(satellite_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val sat_byColumnAndRow = satellite_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val satellite_blockMatrix: BlockMatrix = new CoordinateMatrix(sat_byColumnAndRow).toBlockMatrix()

[Stage 45:===================================================>    (24 + 2) / 26]

satellite_cells_sizeB = Broadcast(50)
satellite_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@1f8ff47e
sat_byColumnAndRow = MapPartitionsRDD[151] at flatMap at <console>:70
satellite_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@1bf52d99


org.apache.spark.mllib.linalg.distributed.BlockMatrix@1bf52d99

### SC

In [17]:
val satellite_M_1_Gc = sc.parallelize(Array[Vector](satellite_summary.mean)).map(m => Vectors.dense(m.toArray))
val satellite_M_1_Gc_RowM: RowMatrix = new RowMatrix(satellite_M_1_Gc)
val sat_M_1_Gc_byColumnAndRow = satellite_M_1_Gc_RowM.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val satellite_M_1_Gc_blockMatrix = new CoordinateMatrix(sat_M_1_Gc_byColumnAndRow).toBlockMatrix()

val sat_matrix_Nt_1 = new Array[Double](satellite_grids.count().toInt)
satellite_grids.unpersist(false)

for (i <- 0 until sat_matrix_Nt_1.length)
  sat_matrix_Nt_1(i) = 1
val satellite_M_Nt_1 = sc.parallelize(sat_matrix_Nt_1).map(m => Vectors.dense(m))
val satellite_M_Nt_1_RowM: RowMatrix = new RowMatrix(satellite_M_Nt_1)
val sat_M_Nt_1_byColumnAndRow = satellite_M_Nt_1_RowM.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val satellite_M_Nt_1_blockMatrix = new CoordinateMatrix(sat_M_Nt_1_byColumnAndRow).toBlockMatrix()
val satellite_M_Nt_Gc_blockMatrix = satellite_M_Nt_1_blockMatrix.multiply(satellite_M_1_Gc_blockMatrix)

val Sc = satellite_blockMatrix.subtract(satellite_M_Nt_Gc_blockMatrix)
Sc.persist(StorageLevel.MEMORY_AND_DISK)

[Stage 54:================================================>         (5 + 1) / 6]

satellite_M_1_Gc = MapPartitionsRDD[157] at map at <console>:76
satellite_M_1_Gc_RowM = org.apache.spark.mllib.linalg.distributed.RowMatrix@b7ebda
sat_M_1_Gc_byColumnAndRow = MapPartitionsRDD[160] at flatMap at <console>:82
satellite_M_1_Gc_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@17b9d603
sat_matrix_Nt_1 = Array(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


satellite_M_Nt_1: org.apache.spark.rdd.RDD[org.apache.spa...


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

### Model Data

In [18]:
//Model
val model_cells_sizeB = sc.broadcast(model_cells_size)
val model_mat: RowMatrix = new RowMatrix(model_grids.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(model_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))

// Split the matrix into one number per line.
val mod_byColumnAndRow = model_mat.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val model_blockMatrix: BlockMatrix = new CoordinateMatrix(mod_byColumnAndRow).transpose().toBlockMatrix()

[Stage 56:====================================================>   (34 + 2) / 36]

model_cells_sizeB = Broadcast(62)
model_mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@69832c0e
mod_byColumnAndRow = MapPartitionsRDD[192] at flatMap at <console>:70
model_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@704ec3f2


org.apache.spark.mllib.linalg.distributed.BlockMatrix@704ec3f2

### Mc

In [19]:
//MC
val model_M_1_Gc = sc.parallelize(Array[Vector](model_summary.mean)).map(m => Vectors.dense(m.toArray))
val model_M_1_Gc_RowM: RowMatrix = new RowMatrix(model_M_1_Gc)
val mod_M_1_Gc_byColumnAndRow = model_M_1_Gc_RowM.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val model_M_1_Gc_blockMatrix = new CoordinateMatrix(mod_M_1_Gc_byColumnAndRow).toBlockMatrix()

val model_matrix_Nt_1 = new Array[Double](model_grids.count().toInt)
model_grids.unpersist(false)

for (i <- 0 until model_matrix_Nt_1.length)
  model_matrix_Nt_1(i) = 1
val model_M_Nt_1 = sc.parallelize(model_matrix_Nt_1).map(m => Vectors.dense(m))
val model_M_Nt_1_RowM: RowMatrix = new RowMatrix(model_M_Nt_1)
val mod_M_Nt_1_byColumnAndRow = model_M_Nt_1_RowM.rows.zipWithIndex.map {
  case (row, rowIndex) => row.toArray.zipWithIndex.map {
    case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
  }
}.flatMap(x => x)
val model_M_Nt_1_blockMatrix = new CoordinateMatrix(mod_M_Nt_1_byColumnAndRow).toBlockMatrix()
val model_M_Nt_Gc_blockMatrix = model_M_Nt_1_blockMatrix.multiply(model_M_1_Gc_blockMatrix)
val model_M_Gc_Nt_blockMatrix = model_M_Nt_Gc_blockMatrix.transpose

[Stage 65:======================================>                   (4 + 2) / 6]

model_M_1_Gc = MapPartitionsRDD[199] at map at <console>:70
model_M_1_Gc_RowM = org.apache.spark.mllib.linalg.distributed.RowMatrix@6c3996a
mod_M_1_Gc_byColumnAndRow = MapPartitionsRDD[202] at flatMap at <console>:76
model_M_1_Gc_blockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@7e406292
model_matrix_Nt_1 = Array(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


model_M_Nt_1: org.apache.spark.rdd.RDD[org.apache.spark.mllib.lina...


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [20]:
val Mc = model_blockMatrix.subtract(model_M_Gc_Nt_blockMatrix)
Mc.persist(StorageLevel.MEMORY_AND_DISK)

Mc = org.apache.spark.mllib.linalg.distributed.BlockMatrix@30924d51


org.apache.spark.mllib.linalg.distributed.BlockMatrix@30924d51

## Matrix Multiplication

In [21]:
//Matrix Multiplication
//val matrix_mul = model_blockMatrix.multiply(satellite_blockMatrix)
val matrix_mul = Mc.multiply(Sc)

[Stage 73:>                                                         (0 + 3) / 3]

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 2 in stage 73.0 failed 4 times, most recent failure: Lost task 2.3 in stage 73.0 (TID 1497, 145.100.59.157, executor 6): java.lang.IllegalArgumentException: requirement failed: The last value of colPtrs must equal the number of elements. values.length: 42531, colPtrs.last: 26624
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.mllib.linalg.SparseMatrix.<init>(Matrices.scala:590)
	at org.apache.spark.mllib.linalg.SparseMatrix.<init>(Matrices.scala:618)
	at org.apache.spark.mllib.linalg.Matrices$.fromBreeze(Matrices.scala:995)
	at org.apache.spark.mllib.linalg.distributed.BlockMatrix$$anonfun$10.apply(BlockMatrix.scala:378)
	at org.apache.spark.mllib.linalg.distributed.BlockMatrix$$anonfun$10.apply(BlockMatrix.scala:365)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:216)
	at org.apache

In [15]:
    println(Mc.numColBlocks + " " + Mc.colsPerBlock)
    println(Mc.numRowBlocks + " " + Mc.rowsPerBlock)

    println(Sc.numColBlocks + " " + Sc.colsPerBlock)
    println(Sc.numRowBlocks + " " + Sc.rowsPerBlock)
    print(num_cols_rows)

1 1024
9879 1024
9879 1024
1 1024
(7808,3892)

lastException: Throwable = null


## SVD

In [17]:
//SVD
val resRowMatrix: RowMatrix = new RowMatrix(matrix_mul.toIndexedRowMatrix().rows.sortBy(_.index).map(_.vector))

val svd: SingularValueDecomposition[RowMatrix, Matrix] = resRowMatrix.computeSVD(10, true)

val U: RowMatrix = svd.U // The U factor is a RowMatrix.
val s: Vector = svd.s // The singular values are stored in a local dense vector.
val V: Matrix = svd.V // The V factor is a local dense matrix.

[Stage 43:==============================================>         (15 + 2) / 18]

resRowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@23f7f254
svd = 


-...


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@7e2ba9a4,[1.7605833863003616E12,5.7253741451916664E10,4.151185732299026E10,3.668228226505551E10,3.4341034557407806E10,2.7715911601627808E10,2.7007353130962887E10,2.508487453875547E10,2.2488917091979183E10,2.102419109157361E10],-0.23613735396347657  0.20480673557119244    ... (10 total)
-0.24143303278185568  -0.18397058442540593   ...
-0.24161245427980513  -0.033384305061724545  ...
-0.21216410577714592  -0.02129411618682879   ...
-0.2665941903718298   0.14562276645638914    ...
-0.2414032554256322   0.17619442814562367    ...
-0.22890794179301527  0.08868200678850471    ...
-0.23675724054458858  0.21530656760113956    ...
-0.25469330957640596  0.18073722214789012    ...
-0.2344741040161406   -0.05979106658508153   ...
-0.2484656543027556   0.1589357811365968     ...
-0.2551123790885418   0.2400536946014573     ...
-0.2301680313794673   0.15426225035416138    ...
-0.2549973419759259   -0.48412263098227865   .

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. The process is not straight forward because the ArrayDouble used for the creation of each dense Vector does not contain the NaN values, therefore there is not a direct between the indices in the Tile's grid and the ones in **kmeans_res** (kmeans result).

To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [18]:
//Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
val multi_res = sc.parallelize(s.toArray).zipWithIndex().map{ case (v,i) => (i,v)}
val multi_cell_pos = multi_res.join(model_grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}).map{ case (k,(v,i)) => (v,i)}

//Associate a Cluster_IDs to respective Grid_cell
val grid_multi = model_grid0.map{ case (i, v) => if (v == -1000) (i,Double.NaN) else (i,v)}.leftOuterJoin(multi_cell_pos.map{ case (c,i) => (i.toLong, c)})

//Convert all None to NaN
val grid_multi_res = grid_multi.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get.toDouble)}

//Define a Tile
val corr_cells :Array[Double] = grid_multi_res.values.collect()

val corr_cellsD = DoubleArrayTile(corr_cells, num_cols_rows._1, num_cols_rows._2)

val geoTif = new SinglebandGeoTiff(corr_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))

//Save to /tmp/
GeoTiffWriter.write(geoTif, corr_tif_tmp)

//Upload to HDFS
var cmd = "hadoop dfs -copyFromLocal -f " + corr_tif_tmp + " " + corr_tif
Process(cmd)!

//Remove from /tmp/
cmd = "rm -fr " + corr_tif_tmp
Process(cmd)!

Name: java.lang.ArrayIndexOutOfBoundsException
Message: 10
StackTrace: java.lang.ArrayIndexOutOfBoundsException: 10
  at geotrellis.raster.DoubleConstantNoDataArrayTile.applyDouble(DoubleArrayTile.scala:125)
  at geotrellis.raster.ArrayTile$class.getDouble(ArrayTile.scala:337)
  at geotrellis.raster.DoubleArrayTile.getDouble(DoubleArrayTile.scala:24)
  at geotrellis.raster.CroppedTile.getDouble(CroppedTile.scala:116)
  at geotrellis.raster.CroppedTile.mutable(CroppedTile.scala:155)
  at geotrellis.raster.CroppedTile.mutable(CroppedTile.scala:133)
  at geotrellis.raster.CroppedTile.toArrayTile(CroppedTile.scala:125)
  at geotrellis.raster.CroppedTile.toBytes(CroppedTile.scala:206)
  at geotrellis.raster.io.geotiff.GeoTiffTile$.apply(GeoTiffTile.scala:103)
  at geotrellis.raster.io.geotiff.package$GeoTiffTileMethods.toGeoTiffTile(package.scala:31)
  at geotrellis.raster.io.geotiff.SinglebandGeoTiff.imageData(SinglebandGeoTiff.scala:40)
  at geotrellis.raster.io.geotiff.writer.GeoTiffWrit

# [Visualize results](plot_kmeans_clusters.ipynb)